In [33]:
import os
import sys
import importlib

import numpy as np
import pandas as pd

sweep_dir = "..\\..\\experiments\\hgd_training\\phase2\\fgsm_cifar10\\sweep_2025-09-14_12-50-27"

In [34]:
def read_test_results_csv(file_path, model_type="normal"):
    """
    Reads the test results from a CSV file and returns a DataFrame.
    """
    try:
        df = pd.read_csv(file_path)
        if model_type != "all":
            df = df[df['Model'] == model_type]
        return df
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return pd.DataFrame()
    except pd.errors.EmptyDataError:
        print(f"File {file_path} is empty.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return pd.DataFrame()
    
model_types = [
    "normal",
    "negative",
    "hybrid normal",
    "hybrid negative",
    "synergy normal",
    "synergy negative",
    "synergy all",
    "synergy trained all"
]
    
# Create dataframe to hold all results and define columns
columns = [
    "job_id",
    "learning rate",
    "bilinear",
    "learn noise",
    "loss type"
]
iter_col = "epsilons"
results_df = pd.DataFrame(columns=columns)
    
dataset_name = None
data = {}

# Go through each directory in the sweep directory
job_dirs = [d for d in os.listdir(sweep_dir) if os.path.isdir(os.path.join(sweep_dir, d))]

for job_dir in job_dirs:
    job_path = os.path.join(sweep_dir, job_dir)
    # load config file
    config_file_path = os.path.join(job_path, "code\\config.py")
    print(f"Loading config from: {config_file_path}")
    module_name = job_dir.replace("-", "_")
    spec = importlib.util.spec_from_file_location(module_name, config_file_path)
    config_module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = config_module
    spec.loader.exec_module(config_module)
    # read test results
    test_results_file = os.path.join(job_path, "results\\results.csv")
    test_results_df = read_test_results_csv(test_results_file, model_type="all")
    test_results_df.columns = test_results_df.columns.str.capitalize()

    if dataset_name is None:
        dataset_name = config_module.dataset_name

    epsilons = config_module.attack_params[config_module.attack_type].get("epsilons", None)
    trained_model = os.path.splitext(os.path.basename(config_module.train_model_paths[0]))[0][6:]
    print(f"Job: {job_dir}, Epsilons: {epsilons}, Trained Model: {trained_model}")

    # lets first create dictionary styled data    
    data[job_dir] = {
        "config": {
            "learning rate": config_module.learning_rate,
            "bilinear": config_module.bilinear,
            "learn noise": config_module.learn_noise,
            "loss type": config_module.loss,
            "trained model": trained_model
        },
        "results": test_results_df.to_dict(orient='records'),
    }
   
print(f"Dataset: {dataset_name}")
for job_id, job_data in data.items():
    print(f"Job ID: {job_id}")
    print("Config:")
    for key, value in job_data["config"].items():
        print(f"  {key}: {value}")
    print("Results:")
    for result in job_data["results"]:
        print(f"  {result}")
    print("\n")

Loading config from: ..\..\experiments\hgd_training\phase2\fgsm_cifar10\sweep_2025-09-14_12-50-27\2025-09-14_12-50-27\code\config.py
Job: 2025-09-14_12-50-27, Epsilons: [0.01, 0.02, 0.03, 0.04, 0.05], Trained Model: normal
Loading config from: ..\..\experiments\hgd_training\phase2\fgsm_cifar10\sweep_2025-09-14_12-50-27\2025-09-14_12-50-28\code\config.py
Job: 2025-09-14_12-50-28, Epsilons: [0.01, 0.02, 0.03, 0.04, 0.05], Trained Model: negative
Loading config from: ..\..\experiments\hgd_training\phase2\fgsm_cifar10\sweep_2025-09-14_12-50-27\2025-09-14_12-50-29\code\config.py
Job: 2025-09-14_12-50-29, Epsilons: [0.01, 0.02, 0.03, 0.04, 0.05], Trained Model: hybrid_nor
Loading config from: ..\..\experiments\hgd_training\phase2\fgsm_cifar10\sweep_2025-09-14_12-50-27\2025-09-14_12-50-30\code\config.py
Job: 2025-09-14_12-50-30, Epsilons: [0.01, 0.02, 0.03, 0.04, 0.05], Trained Model: hybrid_neg
Loading config from: ..\..\experiments\hgd_training\phase2\fgsm_cifar10\sweep_2025-09-14_12-50-27\

In [35]:
def validate_non_denoised_results(data):
    """
    Validates if non-denoised results are consistent across different jobs
    with the same configuration (Model, Epsilon).
    
    Args:
        data (dict): Dictionary containing all jobs data
        
    Returns:
        bool: True if all non-denoised results are consistent, False otherwise
    """
    # Dictionary to store results for comparison
    # Key: (Model, Epsilon), Value: set of Accuracy values
    reference_results = {}
    
    for job_id, job_data in data.items():
        results = job_data["results"]
        
        # Filter for non-denoised results
        non_denoised = [r for r in results if r["Denoised"] == "No"]
        
        # Store results by (Model, Epsilon) combination
        for result in non_denoised:
            key = (result["Model"], result["Epsilon"])
            accuracy = result["Accuracy"]
            
            if key in reference_results:
                reference_results[key].add(accuracy)
            else:
                reference_results[key] = {accuracy}
    
    # Check if all sets have only one value
    inconsistencies = []
    for (model, epsilon), accuracies in reference_results.items():
        if len(accuracies) > 1:
            inconsistencies.append({
                "Model": model,
                "Epsilon": epsilon,
                "Different Accuracies": list(accuracies)
            })
    
    if inconsistencies:
        print("Found inconsistencies in non-denoised results:")
        for inc in inconsistencies:
            print(f"Model: {inc['Model']}, Epsilon: {inc['Epsilon']}")
            print(f"Different accuracy values found: {inc['Different Accuracies']}")
        return False
    
    print("All non-denoised results are consistent across jobs!")
    return True

# Test the validation function
validate_non_denoised_results(data)

All non-denoised results are consistent across jobs!


True

In [36]:
# Create a list to store the processed data
processed_data = {}

# Process each job's data
for job_id, job_data in data.items():
    # Get configuration data
    config = job_data["config"]
    
    # Filter results for normal model and denoised=True
    results = [r for r in job_data["results"] if r["Denoised"] == "Yes"]
    
    row = {}
    for result in results:
        model_type = result['Model']
        if model_type not in row.keys():
            row[model_type] = [result["Accuracy"]]
        else:
            row[model_type].append(result["Accuracy"])

    for model_type, accuracies in row.items():
        row[model_type] = sum(row[model_type]) / len(row[model_type])
    processed_data[config["trained model"]] = row

# Create DataFrame
df = pd.DataFrame(processed_data)
# Display the DataFrame
print("Results DataFrame:")
display(df)

Results DataFrame:


,normal,negative,hybrid_nor,hybrid_neg,synergy_nor,synergy_neg,synergy_all,tr_synergy_all
normal,0.92354,0.90754,0.91336,0.90772,0.91938,0.90842,0.91820,0.92606
negative,0.90382,0.91388,0.89494,0.91860,0.90164,0.91666,0.91376,0.91974
hybrid_nor,0.92362,0.90698,0.91454,0.90940,0.91936,0.91030,0.92022,0.92584
hybrid_neg,0.90300,0.91298,0.89368,0.91560,0.90040,0.91488,0.91272,0.92040
synergy_nor,0.92274,0.90752,0.91290,0.90782,0.91964,0.90876,0.91882,0.92548
synergy_neg,0.90330,0.91412,0.89368,0.91814,0.90184,0.91554,0.91376,0.91824
synergy_all,0.92398,0.92072,0.91440,0.92256,0.92106,0.92034,0.92494,0.93162
tr_synergy_all,0.92214,0.92024,0.91312,0.92180,0.92014,0.92074,0.92456,0.93210


In [37]:
# Create a row for non-denoised results
non_denoised_row = {}

# Get one set of non-denoised results (they should all be the same as verified by validate_non_denoised_results)
first_job_data = next(iter(data.values()))
non_denoised_results = [r for r in first_job_data["results"] if r["Denoised"] == "No"]

non_denoised_row = {}
for result in non_denoised_results:
    model_type = result['Model']
    if model_type not in non_denoised_row.keys():
        non_denoised_row[model_type] = [result["Accuracy"]]
    else:
        non_denoised_row[model_type].append(result["Accuracy"])

for model_type, accuracies in non_denoised_row.items():
    non_denoised_row[model_type] = sum(non_denoised_row[model_type]) / len(non_denoised_row[model_type])

non_denoised_df = pd.DataFrame([non_denoised_row])
print("\nBaseline Results:")
display(non_denoised_df)
# Create final DataFrame with baseline
df_final = pd.concat([
    non_denoised_df,
    df
], ignore_index=False)
df_final.index.values[0] = "no defense"
print("\nTotal Results DataFrame with Baseline:")
display(df_final)


Baseline Results:


,normal,negative,hybrid_nor,hybrid_neg,synergy_nor,synergy_neg,synergy_all,tr_synergy_all
0,0.4047,0.4047,0.39976,0.40434,0.41172,0.41184,0.43148,0.39884



Total Results DataFrame with Baseline:


,normal,negative,hybrid_nor,hybrid_neg,synergy_nor,synergy_neg,synergy_all,tr_synergy_all
no defense,0.40470,0.40470,0.39976,0.40434,0.41172,0.41184,0.43148,0.39884
normal,0.92354,0.90754,0.91336,0.90772,0.91938,0.90842,0.91820,0.92606
negative,0.90382,0.91388,0.89494,0.91860,0.90164,0.91666,0.91376,0.91974
hybrid_nor,0.92362,0.90698,0.91454,0.90940,0.91936,0.91030,0.92022,0.92584
hybrid_neg,0.90300,0.91298,0.89368,0.91560,0.90040,0.91488,0.91272,0.92040
synergy_nor,0.92274,0.90752,0.91290,0.90782,0.91964,0.90876,0.91882,0.92548
synergy_neg,0.90330,0.91412,0.89368,0.91814,0.90184,0.91554,0.91376,0.91824
synergy_all,0.92398,0.92072,0.91440,0.92256,0.92106,0.92034,0.92494,0.93162
tr_synergy_all,0.92214,0.92024,0.91312,0.92180,0.92014,0.92074,0.92456,0.93210


In [38]:
mapping = {
    "normal": "normal",
    "negative": "negative",
    "hybrid_nor": "hybrid normal",
    "hybrid_neg": "hybrid negative",
    "synergy_nor": "synergy normal",
    "synergy_neg": "synergy negative",
    "synergy_all": "synergy all",
    "tr_synergy_all": "synergy trained all"
}
df_final = df_final.rename(columns=mapping, index=mapping)
# Capitalize all words in columns and index
df_final = df_final.rename(columns=lambda x: x.title(), index=lambda x: x.title())
display(df_final)

,Normal,Negative,Hybrid Normal,Hybrid Negative,Synergy Normal,Synergy Negative,Synergy All,Synergy Trained All
No Defense,0.40470,0.40470,0.39976,0.40434,0.41172,0.41184,0.43148,0.39884
Normal,0.92354,0.90754,0.91336,0.90772,0.91938,0.90842,0.91820,0.92606
Negative,0.90382,0.91388,0.89494,0.91860,0.90164,0.91666,0.91376,0.91974
Hybrid Normal,0.92362,0.90698,0.91454,0.90940,0.91936,0.91030,0.92022,0.92584
Hybrid Negative,0.90300,0.91298,0.89368,0.91560,0.90040,0.91488,0.91272,0.92040
Synergy Normal,0.92274,0.90752,0.91290,0.90782,0.91964,0.90876,0.91882,0.92548
Synergy Negative,0.90330,0.91412,0.89368,0.91814,0.90184,0.91554,0.91376,0.91824
Synergy All,0.92398,0.92072,0.91440,0.92256,0.92106,0.92034,0.92494,0.93162
Synergy Trained All,0.92214,0.92024,0.91312,0.92180,0.92014,0.92074,0.92456,0.93210


In [39]:
# Function to bold the maximum value in a series and convert to percentage
def bold_max_percentage(s):
    is_max = s == s.max()
    return [f'\\textbf{{{x*100:.2f}}}' if is_max_val else f'{x*100:.2f}' 
            for x, is_max_val in zip(s, is_max)]

# Apply bold formatting to maximum values in each epsilon column and format the improvement column
formatted_df = df_final.copy()

# Format epsilon columns as percentages
for col in df_final.columns:
    # Convert to numeric, ignoring errors (in case of non-numeric values)
    series = pd.to_numeric(formatted_df[col], errors='coerce')
    formatted_df[col] = bold_max_percentage(series)

# Update the latex_str with the new formatted DataFrame
latex_str = formatted_df.to_latex(
    multirow=True,
    multicolumn=True,
    multicolumn_format='c',
    escape=False,  # Needed to properly render LaTeX bold commands
    index=True  # Remove index column
)

# Save the updated LaTeX table
latex_table_path = os.path.join(sweep_dir, "synergy_results_multicolumn.tex")
with open(latex_table_path, 'w') as f:
    f.write(latex_str)
print(f"LaTeX table saved to {latex_table_path}")

# Display the first few lines of the LaTeX output
print("\nFirst few lines of the LaTeX output:")
print("\n".join(latex_str.split("\n")[:10]))

LaTeX table saved to ..\..\experiments\hgd_training\phase2\fgsm_cifar10\sweep_2025-09-14_12-50-27\synergy_results_multicolumn.tex

First few lines of the LaTeX output:
\begin{tabular}{lllllllll}
\toprule
 & Normal & Negative & Hybrid Normal & Hybrid Negative & Synergy Normal & Synergy Negative & Synergy All & Synergy Trained All \\
\midrule
No Defense & 40.47 & 40.47 & 39.98 & 40.43 & 41.17 & 41.18 & 43.15 & 39.88 \\
Normal & 92.35 & 90.75 & 91.34 & 90.77 & 91.94 & 90.84 & 91.82 & 92.61 \\
Negative & 90.38 & 91.39 & 89.49 & 91.86 & 90.16 & 91.67 & 91.38 & 91.97 \\
Hybrid Normal & 92.36 & 90.70 & \textbf{91.45} & 90.94 & 91.94 & 91.03 & 92.02 & 92.58 \\
Hybrid Negative & 90.30 & 91.30 & 89.37 & 91.56 & 90.04 & 91.49 & 91.27 & 92.04 \\
Synergy Normal & 92.27 & 90.75 & 91.29 & 90.78 & 91.96 & 90.88 & 91.88 & 92.55 \\
